# Copy "Smart Crop Images" into Test

Based on the Test Assignment

Based on Amy's `EDA/Formatting_Data/cleaning_up_s3_data.ipynb` notebook

In [1]:
import pandas as pd
import s3fs
import boto3
import json
from datetime import datetime

# Read Test Reference

In [5]:
df_test = pd.read_csv('test_crops.csv')
df_test.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all
0,141,45,1,0,1,0,0,0,0,0,0
1,141,45,2,0,1,0,0,0,0,0,0
2,141,45,3,0,1,0,0,0,0,0,0
3,141,45,4,0,1,0,0,0,0,0,0
4,141,45,5,0,1,0,0,0,0,0,0


In [6]:
df_test['img_heading_folder'] = df_test['img_id'].astype(str) + '_' + df_test['heading'].astype(str)
df_test.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,img_heading_folder
0,141,45,1,0,1,0,0,0,0,0,0,141_45
1,141,45,2,0,1,0,0,0,0,0,0,141_45
2,141,45,3,0,1,0,0,0,0,0,0,141_45
3,141,45,4,0,1,0,0,0,0,0,0,141_45
4,141,45,5,0,1,0,0,0,0,0,0,141_45


## Create columns for the existing JPG Names and New Names

In [13]:
# New jpg name
df_test['original_image_path'] = 'crops/' + df_test['img_heading_folder'] + '/' + df_test['crop_number'].astype(str) + '.jpg'
df_test['new_image_path'] = 'labeled-test/' + df_test['img_heading_folder'] + '_' + df_test['crop_number'].astype(str) + '.jpg'
df_test.head()

,img_id,heading,crop_number,0_missing,1_null,2_obstacle,3_present,4_surface_prob,5_nosidewalk,6_occlusion,count_all,img_heading_folder,original_image_path,new_image_path
0,141,45,1,0,1,0,0,0,0,0,0,141_45,crops/141_45/1.jpg,labeled-test/141_45_1.jpg
1,141,45,2,0,1,0,0,0,0,0,0,141_45,crops/141_45/2.jpg,labeled-test/141_45_2.jpg
2,141,45,3,0,1,0,0,0,0,0,0,141_45,crops/141_45/3.jpg,labeled-test/141_45_3.jpg
3,141,45,4,0,1,0,0,0,0,0,0,141_45,crops/141_45/4.jpg,labeled-test/141_45_4.jpg
4,141,45,5,0,1,0,0,0,0,0,0,141_45,crops/141_45/5.jpg,labeled-test/141_45_5.jpg


In [22]:
# Grab a sample of images 
#df_test_sample = df_test.iloc[1288:1315]

# Actually Copy files with New Name

In [21]:
# Move files from outer folder to gsv folder
bucket_name = 'gsv-crops2'
s3 = boto3.resource('s3')

start = datetime.now()
print(start)
cannot_move_paths = []
for index, row in df_test.iterrows():
    if index % 500 == 0:
        print(f'Through {index} images at {datetime.now()}')
    # Copy to gsv folder
    old_name = str(row['original_image_path'])
    new_name = str(row['new_image_path'])
    copy_source = {
        'Bucket': bucket_name,
        'Key': old_name
    }
    try:
        s3.meta.client.copy(copy_source, bucket_name, new_name)
    except:
        print(f'Could not move {old_name} at {datetime.now()}')
        cannot_move_paths.append(old_name)
end = datetime.now()
print(end)

2020-03-28 22:47:54.158623
Through 0 images at 2020-03-28 22:47:54.161623
Could not move crops/141_45/1.jpg at 2020-03-28 22:47:54.210415
Could not move crops/141_45/2.jpg at 2020-03-28 22:47:54.220381
Could not move crops/141_45/3.jpg at 2020-03-28 22:47:54.229900
Could not move crops/141_45/4.jpg at 2020-03-28 22:47:54.238906
Could not move crops/141_45/5.jpg at 2020-03-28 22:47:54.248356
Could not move crops/141_45/6.jpg at 2020-03-28 22:47:54.256750
Could not move crops/141_45/7.jpg at 2020-03-28 22:47:54.267485
Could not move crops/141_135/1.jpg at 2020-03-28 22:47:54.278794
Could not move crops/141_135/2.jpg at 2020-03-28 22:47:54.287996
Could not move crops/141_135/3.jpg at 2020-03-28 22:47:54.298248
Could not move crops/141_135/4.jpg at 2020-03-28 22:47:54.311963
Could not move crops/141_135/5.jpg at 2020-03-28 22:47:54.323162
Could not move crops/141_135/6.jpg at 2020-03-28 22:47:54.335854
Could not move crops/141_135/7.jpg at 2020-03-28 22:47:54.347355
Could not move crops/14